In [5]:
import pandas as pd
import numpy as np
from numpy.lib.stride_tricks import as_strided

def calculate_technical_indicators(df: pd.DataFrame, period: int = 14, decay_factor: float = 0.9) -> pd.DataFrame:
    """
    Tính toán các chỉ số kỹ thuật VWAP, WCP, ZLMA, FWMA, và Decay.
    """
    data = df.copy()

    # 1. Weighted Close Price (WCP)
    data['WCP'] = (data['High'] + data['Low'] + 2 * data['Price']) / 4

    # 2. Volume Weighted Average Price (VWAP)
    vwap_num = (data['WCP'] * data['Vol.']).rolling(window=period).sum()
    vwap_den = data['Vol.'].rolling(window=period).sum()
    data['VWAP'] = vwap_num / vwap_den

    # 3. Zero-Lag Moving Average (ZLMA)
    ema1 = data['Price'].ewm(span=period, adjust=False).mean()
    source = 2 * data['Price'] - ema1
    data['ZLMA'] = source.ewm(span=period, adjust=False).mean()

    # 4. Fourier Weight Moving Average (FWMA)
    def fwma_calc(prices: np.ndarray) -> float:
        n = len(prices)
        i = np.arange(1, n + 1)
        weights = np.cos((i - 1) * np.pi / n)
        return np.dot(prices, weights) / np.sum(weights)

    data['FWMA'] = data['Price'].rolling(window=period).apply(fwma_calc, raw=True)

    # 5. Decay
    if len(data) >= period:
        price_vals = data['Price'].values
        vol_vals = data['Vol.'].values

        s_price = price_vals.strides[0]
        price_windows = as_strided(price_vals, shape=(len(price_vals) - period + 1, period), strides=(s_price, s_price))

        s_vol = vol_vals.strides[0]
        vol_windows = as_strided(vol_vals, shape=(len(vol_vals) - period + 1, period), strides=(s_vol, s_vol))

        decay_weights = decay_factor ** np.arange(period - 1, -1, -1)
        numerator = np.sum(price_windows * vol_windows * decay_weights, axis=1)
        denominator = np.sum(vol_windows * decay_weights, axis=1)
        decay_result = np.divide(numerator, denominator, out=np.full_like(numerator, np.nan), where=denominator != 0)
        padded_decay = np.concatenate([np.full(period - 1, np.nan), decay_result])
        data['Decay'] = padded_decay
    else:
        data['Decay'] = np.nan

    return data

# --- BẮT ĐẦU THỰC THI ---
try:
    # 1. Đọc dữ liệu
    df = pd.read_csv('cleaned_stock_data.csv')

    # 2. Tiền xử lý
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values(['Ticker', 'Date'])

    # 3. Áp dụng hàm tính toán
    df_with_indicators = df.groupby('Ticker', group_keys=False).apply(
        calculate_technical_indicators, 
        period=20,
        decay_factor=0.95
    )
    
    # 4. XUẤT RA TỆP CSV
    output_filename = 'stock_data_with_indicators.csv'
    df_with_indicators.to_csv(output_filename, index=False, encoding='utf-8')

    print(f"Hoàn thành! Dữ liệu đã được xuất thành công ra tệp '{output_filename}'.")

except FileNotFoundError:
    print("Lỗi: Không tìm thấy tệp 'cleaned_stock_data.csv'.")
    print("Vui lòng đảm bảo tệp dữ liệu nằm cùng thư mục với file code.")

Hoàn thành! Dữ liệu đã được xuất thành công ra tệp 'stock_data_with_indicators.csv'.


/tmp/ipykernel_572/4008130362.py:65: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_with_indicators = df.groupby('Ticker', group_keys=False).apply(
